The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [6]:

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(URL)

In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


In [11]:
launch_sites_df.dtypes

Launch Site     object
Lat            float64
Long           float64
dtype: object

In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)


In [15]:
m = folium.Map(location=[28.5, -80.5], zoom_start=5)

# Loop through each row of the DataFrame
for _, row in launch_sites_df.iterrows():
    lat = row["Lat"]
    long = row["Long"]
    label = row["Launch Site"]
    popup_text = row["Launch Site"]

    # Add circle
    folium.Circle(
        location=[lat, long],
        radius=1000,  # in meters
        color='#000000',
        fill=True,
        fill_opacity=0.4
    ).add_child(folium.Popup(popup_text)).add_to(m)

    # Add marker with text label
    folium.map.Marker(
        [lat, long],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12pt; color:#d35400;"><b>{label}</b></div>',
        )
    ).add_to(m)

# Display the map
m

In [16]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [20]:

# Create base map
site_map = folium.Map(location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()], zoom_start=5)

# Create marker cluster
marker_cluster = MarkerCluster().add_to(site_map)

# Iterate through each launch record
for index, record in spacex_df.iterrows():
    # Extract coordinates
    coordinates = [record['Lat'], record['Long']]

    # Determine marker color
    marker_color = 'green' if record['class'] == 1 else 'red'

    # Create marker without Flight Number
    marker = folium.Marker(
        location=coordinates,
        popup=f"Launch Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class']==1 else 'Failure'}",
        icon=folium.Icon(color=marker_color, icon_color='white')
    )

    # Add marker to cluster
    marker.add_to(marker_cluster)

# Add marker cluster to map
site_map.add_child(marker_cluster)

# Display map
site_map


In [21]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [22]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [24]:
# Base map
site_map = folium.Map(location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()], zoom_start=6)

# Create marker cluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers for launches
for index, record in spacex_df.iterrows():
    coordinates = [record['Lat'], record['Long']]
    marker_color = 'green' if record['class'] == 1 else 'red'

    marker = folium.Marker(
        location=coordinates,
        popup=f"Launch Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class']==1 else 'Failure'}",
        icon=folium.Icon(color=marker_color)
    )
    marker.add_to(marker_cluster)

# Example: Add line between launch site and closest coastline
# Coordinates of launch site
launch_site_lat = spacex_df.iloc[0]['Lat']
launch_site_lon = spacex_df.iloc[0]['Long']

# Replace these with coordinates of closest coastline
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Add marker at coastline
folium.Marker(
    [coastline_lat, coastline_lon],
    icon=folium.Icon(color='blue'),
    popup=f"Coastline Point ({distance_coastline:.2f} km from Launch Site)"
).add_to(site_map)

# Draw line between launch site and coastline
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color='blue',
    weight=2
).add_to(site_map)

# Add distance label
folium.Marker(
    [(launch_site_lat + coastline_lat)/2, (launch_site_lon + coastline_lon)/2],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12pt; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>',
    )
).add_to(site_map)

# Add Mouse Position plugin
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# Show map
site_map